# pynhm sagehen

## Coupling of MODFLOW to pynhm

## Supported operating systems
This example can be run on the following operating systems:

* linux
* macOS
* Windows

## Prerequisites
To process the results, the following publicly available software are required:

* __flopy__ is a python package that can be used to build, run, and post-process MODFLOW 6 models. The source is available at https://github.com/modflowpy/flopy and the package can be installed from PyPI using `pip install flopy` or conda using `conda install flopy`.
* __geopandas__ which can be installed using PyPI (`pip install geopandas`) or conda (`conda install geopandas`).
* __fiona__ which can be installed using PyPI (`pip install fiona`) or conda (`conda install fiona`).
* __netCDF4__ which can be installed using PyPI (`pip install netCDF4`) or conda (`conda install netCDF4`).

## Post-processing the results

We start by importing the necessary packages:

In [ ]:
import os
import sys
import pathlib as pl

import numpy as np
import matplotlib as mpl
from matplotlib import gridspec
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
import flopy
import geopandas as gpd
import fiona
import netCDF4 as nc

In [ ]:
# Set this to your location for the mf6bmipaper repo
root_dir = pl.Path('/Users/jamesmcc/usgs/mf6bmipaper/models/ModflowPynhm/')
os.chdir(root_dir)

sys.path.append(os.path.join("..", "common"))
from figspecs import USGSFigure

#### Set figure width and set figure specifications

In [ ]:
figwidth = 90 # mm
figwidth = figwidth / 10 / 2.54 # inches

fs = USGSFigure(figure_type="map")

#### Function to map HRU values to MODFLOW 6 values

In [ ]:
def hru2mf6(weights, values):
    return weights.dot(values)

#### Functions to map cell-by-cell data to the grid

In [ ]:
def CellBudgetSum(shape, idx, v, mask=False):
    v_unique = np.zeros(shape, dtype=v.dtype)
    np.add.at(v_unique, idx, v)
    if mask:
        mask_arr = np.ones(shape, dtype=bool)
        mask_arr[np.unique(idx)] = False
        v_unique = np.ma.masked_where(mask_arr, v_unique)
    return v_unique

In [ ]:
def CellBudgetReshape(modelgrid, idx, v, mask=False):
    return CellBudgetSum(modelgrid.nnodes, idx, v, mask=mask).reshape(modelgrid.shape)

#### Process the geodatabase

In [ ]:
file = "../ModflowPRMS/Sagehen.gdb"
hru = gpd.read_file(file, driver="FileGDB", layer="HRU")
river = gpd.read_file(file, driver="FileGDB", layer="stream")

In [ ]:
ws = "sagehenmodel"
sim = flopy.mf6.MFSimulation().load(sim_ws=ws)
gwf = sim.get_model("sagehenmodel")

##### Set coordinate information for model grid

In [ ]:
gwf.modelgrid.set_coord_info(xoff=214860, yoff=4365620, epsg=26911, angrot=12.013768668935385975)

##### Print model discretization shape

In [ ]:
gwf.modelgrid.shape, gwf.modelgrid.nnodes

#### Get PRMS output

In [ ]:
fpth = "sagehenmodel/output/pynhm_output.npz"
prms_out = np.load(fpth)

#### Get prms output times

In [ ]:
# the infiltration and recharge maps show WY 1993 - 1996
# or is this a typo and it's supposed to be 1982- instead of 1992-?
from datetime import datetime
time_end = datetime(1996,10,1)
time_start = datetime(1980, 10, 1)
print(time_end - time_start)

map_endish_idx0 = datetime(1992, 10, 1)
print(map_endish_idx0 - time_start)

idx0 = 4383 #0 #730 #365
times = prms_out["time"][idx0:]
ndays = times.shape[0]
print("Number of PRMS days to process {}".format(ndays))



#### Get MODFLOW output times

In [ ]:
tobj = flopy.utils.CellBudgetFile("sagehenmodel/output/gwf_sagehen-gsf.sfr.cbc", precision="double")
times = np.array(tobj.get_times())[idx0:]
ndays = times.shape[0]
print("Number of MODFLOW days to process {}".format(ndays))

##### Calculate cell area and conversion factors

In [ ]:
cell_area = 90. * 90.
cum2m = 1. / cell_area
m2mm = 1000.
cum2mpd = cum2m / ndays
cum2mmpd = cum2mpd * m2mm

m2ft = 3.28081
in2m = 1. / (12. * m2ft)

##### Get idomain for mapping

In [ ]:
idomain = gwf.dis.idomain.array

#### Get HRU areas and convert to square meters

In [ ]:
param_file = 'sagehen_params.pkl'
import pickle
with open(param_file, "rb") as input_file:
     param_dict = pickle.load(input_file)
import pywatershed
params = pywatershed.PrmsParameters(param_dict)

hru_area = params.parameters["hru_area"][:] # m2
acre2m2 = 43560. / (m2ft * m2ft)
hru_area *= acre2m2

##### Calculate model area

In [ ]:
active_area = cell_area * idomain[0, :, :].sum()
active_area

##### Get uzf mapping arrays

In [ ]:
tobj = flopy.utils.CellBudgetFile("sagehenmodel/output/gwf_sagehen-gsf.uzf.cbc", precision="double")
v = tobj.get_data(totim=times[0], text="GWF")[0]
uzf_nodes = v["node"] - 1
uzf_gwfnodes = v["node2"] - 1

In [ ]:
print(uzf_nodes, "\n", uzf_gwfnodes)

#### Read weights

The weight matrix should have columns equal to the number of HRUs and rows equal to the number of UZF cells or number of SFR reaches.

_UZF weights_

In [ ]:
uz2 = np.load('weights.npz')
print(uz2['uzfw'].shape, uz2['sfrw'].shape)
uzfw = uz2['uzfw']

_Number of UZF cells at the top of the model_

In [ ]:
nuzf_infilt = uzfw.shape[0]
print("number of UZF cells at the top of the model {}".format(nuzf_infilt))

#### Process sfr budget output

In [ ]:
fpth = "sagehenmodel/output/gwf_sagehen-gsf.sfr.cbc"
sfrobj = flopy.utils.CellBudgetFile(fpth, precision="double")
sfrobj.get_unique_record_names(0)

In [ ]:
ext_outflow = 0.
for totim in times:
    ext_outflow -= sfrobj.get_data(totim=totim, text="EXT-OUTFLOW")[0]["q"][-1]
ext_outflow

#### Plot the model grid with the HRUs

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
mm.plot_inactive(zorder=70)
mm.plot_grid(lw=0.25, color="0.5", zorder=10)
river.plot(ax=mm.ax, color="cyan", lw=1, zorder=101)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5, zorder=100);
fpth = os.path.join("..", "..", "doc", "figures", "sagehen_pynhm_all_discretization.png")
plt.savefig(fpth, dpi=600);

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5, zorder=100);
fpth = os.path.join("..", "..", "doc", "figures", "sagehen_pynhm_hrus.png")
plt.savefig(fpth, dpi=600);

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
mm.plot_inactive(zorder=70)
river.plot(ax=mm.ax, color="cyan", lw=1, zorder=101)
fpth = os.path.join("..", "..", "doc", "figures", "sagehen_pynhm_sfr_network.png")
plt.savefig(fpth, dpi=600);

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
mm.plot_inactive(zorder=70)
mm.plot_grid(lw=0.25, color="0.5", zorder=10)
fpth = os.path.join("..", "..", "doc", "figures", "sagehen_pynhm_gw_grid.png")
plt.savefig(fpth, dpi=600);

#### Process the GWF cbc file

In [ ]:
cbcobj = flopy.utils.CellBudgetFile("sagehenmodel/output/gwf_sagehen-gsf.cbc", precision="double")
cbcobj.get_unique_record_names()

##### Groundwater recharge, evapotranspiration, and storage changes

In [ ]:
for idx, totim in enumerate(times):
    v = cbcobj.get_data(totim=totim, text="UZF-GWRCH")[0]
    u = cbcobj.get_data(totim=totim, text="UZF-GWET")[0]
    sy = cbcobj.get_data(totim=totim, text="STO-SY")[0]
    ss = cbcobj.get_data(totim=totim, text="STO-SS")[0]
    if idx == 0:
        gwrch_gwfnodes = v["node"] - 1
        gwrch_uzfnodes = v["node2"] - 1
        gwrch_tot = np.zeros(v["q"].shape, dtype=v["q"].dtype)
        gwet_tot = np.zeros(u["q"].shape, dtype=u["q"].dtype)
        gwsto_tot = np.zeros(sy.shape, dtype=sy.dtype)
    gwrch_tot += v["q"]
    gwet_tot -= u["q"]
    gwsto_tot -= sy
    gwsto_tot -= ss
gwrch_tot *= cum2mmpd
gwet_tot *= cum2mmpd
gwsto_tot *= cum2mmpd

In [ ]:
gwrch_tot = CellBudgetReshape(gwf.modelgrid, gwrch_gwfnodes, gwrch_tot, mask=True)
gwrch_tot = np.ma.masked_where(idomain==0, gwrch_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(gwrch_tot)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="white", facecolor="none", lw=0.5)
plt.colorbar(v);

#### Groundwater evapotranspiration

In [ ]:
gwet_tot = CellBudgetReshape(gwf.modelgrid, gwrch_gwfnodes, gwet_tot, mask=True)
gwet_tot = np.ma.masked_where(idomain==0, gwet_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(gwet_tot)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="white", facecolor="none", lw=0.5)
plt.colorbar(v);

##### Groundwater discharge to the surface

In [ ]:
for idx, totim in enumerate(times):
    v = cbcobj.get_data(totim=totim, text="DRN-TO-MVR")[0]
    if idx == 0:
        gwd_tot = np.zeros(v["q"].shape, dtype=v["q"].dtype)
        gwd_gwfnodes = v["node"] - 1
        gwd_nodes2 = v["node2"] - 1
    gwd_tot -= v["q"]
gwd_tot *= cum2mmpd

In [ ]:
gwd_tot = CellBudgetReshape(gwf.modelgrid, gwd_gwfnodes, gwd_tot, mask=True)
gwd_tot = np.ma.masked_where(gwd_tot==0., gwd_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(gwd_tot, vmax=0.08)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v)

##### Boundary conditions

In [ ]:
for idx, totim in enumerate(times):
    v = cbcobj.get_data(totim=totim, text="CHD")[0]
    if idx == 0:
        gwchd_gwfnodes = v["node"] - 1
        print(gwchd_gwfnodes)
        gwchd_tot = np.zeros(v["q"].shape, dtype=v["q"].dtype)
    gwchd_tot += v["q"]
gwchd_tot *= cum2mmpd


In [ ]:
gwchd_tot = CellBudgetReshape(gwf.modelgrid, gwchd_gwfnodes, gwchd_tot, mask=True)
gwchd_tot = np.ma.masked_where(gwchd_tot==0., gwchd_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(gwchd_tot, vmax=0.08)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v)

#### Groundwater storage changes

In [ ]:
gwsto_tot = np.ma.masked_where(idomain==0, gwsto_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(gwsto_tot)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v)

#### Process the UZF cbc file

In [ ]:
uzfobj = flopy.utils.CellBudgetFile("sagehenmodel/output/gwf_sagehen-gsf.uzf.cbc", precision="double")
uzfobj.get_unique_record_names()

In [ ]:
for idx, totim in enumerate(times):
    v = uzfobj.get_data(totim=totim, text="GWF")[0]
    v2 = uzfobj.get_data(totim=totim, text="INFILTRATION")[0]
    v3 = uzfobj.get_data(totim=totim, text="REJ-INF-TO-MVR")[0]
    u = uzfobj.get_data(totim=totim, text="UZET")[0]
    s = uzfobj.get_data(totim=totim, text="STORAGE")[0]
    if idx == 0:
        uzfinf_tot = np.zeros(v2["q"].shape, dtype=v2["q"].dtype)
        uzfrinf_tot = np.zeros(v3["q"].shape, dtype=v3["q"].dtype)
        uzfrch_tot = np.zeros(v["q"].shape, dtype=v["q"].dtype)
        uzfet_tot = np.zeros(u["q"].shape, dtype=u["q"].dtype)
        uzfsto_tot = np.zeros(s["q"].shape, dtype=s["q"].dtype)
    uzfinf_tot += v2["q"] + v3["q"]
    uzfrinf_tot -= v3["q"]
    uzfrch_tot -= v["q"]
    uzfet_tot -= u["q"]
    uzfsto_tot -= s["q"]
uzfrch_tot *= cum2mmpd
uzfinf_tot *= cum2mmpd
uzfrinf_tot *= cum2mmpd
uzfet_tot *= cum2mmpd
uzfsto_tot *= cum2mmpd

##### Create 2d idomain array

In [ ]:
idomain_2d = np.sum(idomain, axis=0)

##### Create 2d infiltration array

In [ ]:
v = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, uzfinf_tot, mask=True)
uzf_inf2d = np.sum(v, axis=0)
uzf_inf2d = np.ma.masked_where(idomain_2d==0, uzf_inf2d)

In [ ]:
uzf_inf2d.shape, uzf_inf2d.min(), uzf_inf2d.mean(), uzf_inf2d.max()

##### Create 2d recharge array

In [ ]:
v = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, uzfrch_tot, mask=True)
# uzf_rch2d = v[0, :, :] + v[1, : , :]
uzf_rch2d = np.sum(v, axis=0)
print(uzf_rch2d.shape)
uzf_rch2d = np.ma.masked_where(idomain_2d==0, uzf_rch2d)

In [ ]:
uzf_rch2d.shape, uzf_rch2d.min(), uzf_rch2d.mean(), uzf_rch2d.max()

In [ ]:
idomain.shape

##### Groundwater recharge from UZF cell-by-cell

In [ ]:
uzfrch_tot = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, uzfrch_tot, mask=True)
uzfrch_tot = np.ma.masked_where(idomain==0, uzfrch_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(uzfrch_tot)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

##### UZF infiltration from UZF cell-by-cell

In [ ]:
uzfinf_tot = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, uzfinf_tot, mask=True)
uzfinf_tot = np.ma.masked_where(idomain==0, uzfinf_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(uzfinf_tot)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="white", facecolor="none", lw=0.5)
plt.colorbar(v);

##### UZF rejected infiltration

In [ ]:
uzfrinf_tot = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, uzfrinf_tot, mask=True)
uzfrinf_tot = np.ma.masked_where(uzfrinf_tot==0., uzfrinf_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(uzfrinf_tot, vmax=0.08)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

##### Rejected infiltration and drainage

In [ ]:
for idx, totim in enumerate(times):
    v = cbcobj.get_data(totim=totim, text="DRN-TO-MVR")[0]
    if idx == 0:
        gwd_tot = np.zeros(v["q"].shape, dtype=v["q"].dtype)
        gwd_gwfnodes = v["node"] - 1
        gwd_nodes2 = v["node2"] - 1
    gwd_tot -= v["q"]
    v2 = uzfobj.get_data(totim=totim, text="REJ-INF-TO-MVR")[0]
    if idx == 0:
        uzfrinf_tot = np.zeros(v2["q"].shape, dtype=v2["q"].dtype)
        vv = uzfobj.get_data(totim=totim, text="GWF")[0]
    uzfrinf_tot -= v2["q"]
gwd_tot *= cum2mmpd
uzfrinf_tot *= cum2mmpd

In [ ]:
gwd_tot = CellBudgetReshape(gwf.modelgrid, gwd_gwfnodes, gwd_tot, mask=True)
uzfrinf_tot = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, uzfrinf_tot, mask=True)
gwfro_tot = gwd_tot + uzfrinf_tot
gwfro_tot = np.ma.masked_where(gwfro_tot==0., gwfro_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(gwfro_tot, vmax=0.16)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

#### Unsaturated zone ET

In [ ]:
uzfet_tot = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, uzfet_tot, mask=True)
uzfet_tot = np.ma.masked_where(idomain==0, uzfet_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(uzfet_tot)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

#### Unsaturated zone storage changes

In [ ]:
uzfsto_tot = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, uzfsto_tot, mask=True)
uzfsto_tot = np.ma.masked_where(idomain==0, uzfsto_tot)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(uzfsto_tot)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

#### Map PRMS data to MODFLOW grid

In [ ]:
list(prms_out.keys())

In [ ]:
prms_out["ppt"].shape

#### Function to map PRMS data to the MODFLOW 6 grid

In [ ]:
def prmstomf6(v_prms):
    v_prms /= hru_area
    v_map = hru2mf6(uzfw, v_prms) * cell_area * cum2mmpd
    v_mf6 = np.zeros((v_map.shape[0] + v_map.shape[0]), dtype=v_map.dtype)
    v_mf6[:v_map.shape[0]] = v_map
    v_mf6 = CellBudgetReshape(gwf.modelgrid, uzf_gwfnodes, v_mf6, mask=True)
    return np.ma.masked_where(gwf.dis.idomain.array==0, v_mf6)

#### Map PRMS data to the MODFLOW 6 grid

In [ ]:
ppt_mf6 = prmstomf6(np.sum(prms_out["ppt"], axis=0))
potet_mf6 = prmstomf6(np.sum(prms_out["potet"], axis=0))
actet_mf6 = prmstomf6(np.sum(prms_out["actet"], axis=0))
soil_mf6 = prmstomf6(np.sum(prms_out["infil"], axis=0))

##### PRMS Plot Precipitation

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(ppt_mf6)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(potet_mf6)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(actet_mf6)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(ppt_mf6-actet_mf6)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(soil_mf6)
river.plot(ax=mm.ax, color="cyan", lw=1)
hru.plot(ax=mm.ax, edgecolor="green", facecolor="none", lw=0.5)
plt.colorbar(v);

#### Make report figure

In [ ]:
def add_colorbar(ax, v, label, vmax=16, vscale=1000.):
    cbar = plt.colorbar(v, orientation="horizontal", ax=ax, shrink=0.7)
    ticks = np.linspace(0, vmax, 5)
    cbar.set_ticks(ticks)
    ticks_loc = cbar.ax.get_xticks().tolist()
    cbar.ax.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
    cbar.ax.set_xticklabels(["{:g}".format(x / vscale) for x in ticks])
    cbar.set_label(label)
    return cbar

In [ ]:
def add_legend(ax):
    # add dummy data to axis
    ax.plot([0,0], [1,0], 
            lw=1, 
            color="cyan", 
            label="Sagehen Creek")
    ax.plot(
        [0], [0], 
        lw=0.0,
        marker="s", 
        markeredgecolor=colors[0],
        markerfacecolor="none",
        label="Hydrologic\nResponse\nUnit"
    )
    leg = fs.graph_legend(ax=ax, loc="upper left")
    return leg

In [ ]:
figheight = figwidth * 2
# vmax, vscale = .016, 1.
fig, axes = plt.subplots(
    figsize=(figwidth, figheight),
    ncols=1, 
    nrows=2,
    constrained_layout=True,
)

colors = ("#c36f31", "#cab39f", "#b7bf5e")

ax = axes[0]
vmax, vscale = .04, 1.
mm = flopy.plot.PlotMapView(model=gwf, ax=ax)
v = mm.plot_array(uzf_inf2d*vscale, vmax=vmax, cmap="viridis_r")
river.plot(ax=ax, color="cyan", lw=1)
hru.plot(ax=ax, edgecolor=colors[0], facecolor="none", lw=0.5)
ax.yaxis.offsetText.set_visible(False)
offset = r"$\times$ 10$^6$" #ax.yaxis.get_major_formatter().get_offset()
ax.set_xlabel("x-coordinate (m)")
ax.set_ylabel("y-coordinate ({} m)".format(offset))
fs.heading(ax=ax, idx=0)
cbar = add_colorbar(ax, v, "Average infiltration rate (mm/d)", vmax=vmax, vscale=vscale)
leg = add_legend(ax)

ax = axes[1]
vmax, vscale = .16, 1.
mm = flopy.plot.PlotMapView(model=gwf, ax=ax)
v = mm.plot_array(uzf_rch2d*vscale, vmax=vmax, cmap="viridis_r")
river.plot(ax=ax, color="cyan", lw=1)
hru.plot(ax=ax, edgecolor=colors[0], facecolor="none", lw=0.5)
ax.yaxis.offsetText.set_visible(False)
# offset = ax.yaxis.get_major_formatter().get_offset()
ax.set_xlabel("x-coordinate (m)")
ax.set_ylabel("y-coordinate ({} m)".format(offset))
fs.heading(ax=ax, idx=1)
cbar = add_colorbar(ax, v, "Average groundwater recharge rate (mm/d)", vmax=vmax, vscale=vscale)
leg = add_legend(ax)


fpth = os.path.join("..", "..", "doc", "figures", "sagehen_maps_pywatershed.png")
plt.savefig(fpth, dpi=600);

In [ ]:
v = uzf_inf2d.mean() / m2mm
print("Infiltration {} m/d {} m/y {:,.4f} m3".format(
    v, 
    v * 365.25,
    v * ndays * active_area,
)
)

In [ ]:
v = uzf_rch2d.mean() / m2mm
print("Groundwater recharge {} m/d {} m/y {:,.4f} m3".format(
    v, 
    v * 365.25,
    v * ndays * active_area,
)
)